# Compare similar flights to each other

In [2]:
## First import general packages for running python analysis:
import os, h5py, datetime,pytz
import numpy as np
from matplotlib.pyplot import *
from matplotlib import pyplot as plt
import pickle, glob

from scipy.interpolate import griddata
import scipy.optimize as opt
from scipy.optimize import curve_fit
from scipy.optimize import leastsq
from scipy.optimize import least_squares

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.ticker import MultipleLocator

## Then import the beamcals module packages and initialize 'gbosite' class:
from beamcals import corr
from beamcals import concat
from beamcals import drone
from beamcals import bicolog
import beamcals.plotting_utils as pu
import beamcals.fitting_utils as fu
import beamcals.geometry_utils as gu
import beamcals.time_utils as tu
from beamcals.sites import site
gbosite=site.site('../beamcals/beamcals/sites/GBO_config.npz')

# various gridding attempts
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging

freqs = 800.0*np.ones(1024) + (-400/1024.)*np.arange(1024)


In [ ]:
def get_newpointing(x,y,xc,yx,theta):
    xp = x - xc
    yp = y - yc
    xpp = np.cos(theta)*xp - np.sin(theta)*yp
    ypp = np.sin(theta)*xp + np.cos(theta)*yp
    return xpp, ypp

def get_slice(x,y,ty,val,tol):
    if ty=='x':
        ni = np.where(np.abs(x-val)<=tol)[0]
    if ty=='y':
        ni = np.where(np.abs(y-val)<=tol)[0]
    return ni

def Gauss_2d_cent(x,y,A,xsig,ysig,off):
    xx = ((x)**2)/(2*(xsig**2))
    yy = ((y)**2)/(2*(ysig**2))
    return A*np.exp(-1.0*(xx + yy))+off

def radial_profile(x,y,z,x0,y0):
    r = np.sqrt((x-x0)**2 + (y-y0)**2)
    r = r.astype(np.int)

    tbin = np.bincount(r.ravel(), z.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    return radialprofile 

In [ ]:
## Define anything I want to keep common
dotsize=1
freq_i = 900
res = 500

LX,LY = np.meshgrid(np.linspace(-100,100,res), np.linspace(-100,100,res))
X = np.arange(-100,100,2.0,dtype='float64')
Y = np.arange(-100,100,2.0,dtype='float64')
d=0
# this may change per flight

Epols = [0,2,4,6,8,10,12,14]
Npols = [1,3,5,7,9,11,13,15]

dishes = [0,2,4]
Nf = 1024
Nd = len(dishes)

In [ ]:
fitdir='/hirax/GBO_Analysis_Outputs/main_beam_fits/'
pckldir = '/hirax/GBO_Analysis_Outputs/flight_pickles/'

In [ ]:
flies = ['533','534','535','536','618','619','623','625','648','649','646','647','620','624']

fly = flies[10]

ffile = glob.glob(fitdir+'*'+fly+'*')[0]
pklfile = glob.glob(pckldir+'*'+fly+'*')[0]
print(ffile)
print(pklfile)

pols = Npols
cpols = Epols

# 1. Look at a single flight and examine fits

In [ ]:
#######################
#### Gaussian fit #####
## check amplitudes ###
#######################


markers=['s',',']
msize=[1,3]

fits = np.load(ffile)

D = len(fits['G_popt'][:,0,0])
cm = plt.get_cmap('Spectral')
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.set_prop_cycle(color=[cm(1.*i/D) for i in range(D)])


for d in np.arange(0,D):
    if d in Npols: 
        plt.plot(freqs,fits['G_popt'][d,:,0],marker=markers[0],ms=msize[0],linestyle='None',label=str(d))
    else: 
        plt.plot(freqs,fits['G_popt'][d,:,0],marker=markers[1],ms=msize[1],linestyle='None',label=str(d))
plt.ylim(0,4E-7)
plt.ylabel('Amp [arb]')
plt.xlabel('Freq [MHz]')
plt.legend(markerscale=3)
plt.show()



In [ ]:
## Investigating North pol flights only, let's plot only the co-pol fit values

for d in pols:
    fig, axs = plt.subplots(6,1,figsize=(10,10))
    
    axs[0].plot(freqs,fits['G_popt'][d,:,1],'b,')
    axs[0].set_ylim(-20,20)
    axs[0].set_ylabel('X centroid')
        
    axs[1].plot(freqs,fits['G_popt'][d,:,3],'b,')
    axs[1].set_ylim(-20,20)
    axs[1].set_ylabel('Y centroid')

    axs[2].plot(freqs,fits['G_popt'][d,:,2],'b,')
    axs[2].set_ylim(0,20)
    axs[2].set_ylabel('Xsig')
        
    axs[3].plot(freqs,fits['G_popt'][d,:,4],'b,')
    axs[3].set_ylim(0,20)
    axs[3].set_ylabel('Ysig')

    axs[4].plot(freqs,fits['G_popt'][d,:,0],'b,')
    axs[4].set_ylim(0,4E-7)
    axs[4].set_ylabel('Amp')

    axs[5].plot(freqs,fits['G_popt'][d,:,5],'b,')
    axs[5].set_ylim(0,2E-8)
    axs[5].set_ylabel('offset [arb]')
    fig.tight_layout()
    plt.show()


In [ ]:
## Amplitude to Background ratios:

fig = plt.figure(figsize=(16,8))
for d in pols: 
    plt.plot(freqs,fits['G_popt'][d,:,0]/fits['G_popt'][d,:,5],marker='.',linestyle='None',label=str(d))
    #plt.ylim(0,2E-8)
    plt.ylabel('Amplitude /offset')
    plt.xlabel('Frequency [MHz] ')
    plt.ylim(0,20)
plt.legend()
plt.show()

fig = plt.figure(figsize=(16,8))
for d in pols: 
    plt.plot(freqs,fits['G_popt'][d,:,5],marker='.',linestyle='None',label=str(d))
    #plt.ylim(0,2E-8)
    plt.ylabel('Offset')
    plt.xlabel('Frequency [MHz] ')
    plt.ylim(0,2E-8)
    #plt.xlim(200,300,750)
plt.legend()
plt.show()

# 2. Examine Maps


In [ ]:
#######################
##### Map Making ######
## check amplitudes ###
#######################

with open(pklfile, 'rb') as inp:
    pconcat = pickle.load(inp)

inds_on_cut=np.intersect1d(np.arange(len(pconcat.t_arr_datetime)),pconcat.inds_on).tolist()
indsuse0 = [j for j in inds_on_cut if str(pconcat.drone_xyz_LC_interp[j,0]) != 'nan']

inds_off_cut=np.intersect1d(np.arange(len(pconcat.t_arr_datetime)),pconcat.inds_off).tolist()
indsoff0 = [j for j in inds_off_cut if str(pconcat.drone_xyz_LC_interp[j,0]) != 'nan']


x = pconcat.drone_xyz_LC_interp[indsuse0,0]
y = pconcat.drone_xyz_LC_interp[indsuse0,1]

In [ ]:
LZs = np.zeros([len(pols),len(LX[:,0]),len(LY[0,:])])
RPs = np.zeros([len(pols),142])

for di in range(0,len(pols)):
    d = pols[di]
    xc = pconcat.G_popt[d,freq_i,1] # x centroid
    yc = pconcat.G_popt[d,freq_i,3] # y centroid
    amp_corr = pconcat.V_bgsub[indsuse0,freq_i,d]/pconcat.G_popt[d,freq_i,0]
                
    #plt.scatter(x,y,s=dotsize, c=amp_corr, cmap='gnuplot2')
    #plt.colorbar()
    #plt.show()  

    xp,yp = get_newpointing(x,y,xc,yc,0.0)       
   
    plt.title('INPUT: '+str(d))
    plt.scatter(xp,yp,s=dotsize, c=10*np.log10(amp_corr), cmap='gnuplot2')
    plt.colorbar()
    plt.show()

    # linear gridding, for reference
    try:
        LZs[di,:,:] = griddata((xp,yp), amp_corr, (LX,LY), method='linear')
        RPs[di,:] = radial_profile(LX,LY,LZs[di,:,:],0,0)
    except: 'Could not linearly interpolate!!!!!'

for di in range(0,len(pols)):
    d = pols[di]
    plt.title('INPUT: '+str(d))
    plt.pcolormesh(LX,LY, 10*np.log10(LZs[di,:,:]), shading='auto',cmap='gnuplot2',vmin=0,vmax=-60)
    plt.legend()
    plt.colorbar()
    plt.axis("equal")
    plt.show()    

In [ ]:
## Look at cross-pol instead

LZs = np.zeros([len(pols),len(LX[:,0]),len(LY[0,:])])

for di in range(0,len(pols)):
    d = cpols[di]
    xc = pconcat.G_popt[d,freq_i,1] # x centroid
    yc = pconcat.G_popt[d,freq_i,3] # y centroid
    amp_corr = pconcat.V_bgsub[indsuse0,freq_i,d]/pconcat.G_popt[d,freq_i,0]

    xp,yp = get_newpointing(x,y,xc,yc,0.0)       
    plt.title('INPUT: '+str(d))
    plt.scatter(xp,yp,s=dotsize, c=10*np.log10(amp_corr), cmap='gnuplot2')
    plt.colorbar()
    plt.show()

    # linear gridding, for reference
    try:
        LZs[di,:,:] = griddata((xp,yp), amp_corr, (LX,LY), method='linear')
    except: 'Could not linearly interpolate!!!!!'

for di in range(0,len(pols)):
    d = cpols[di]
    plt.title('INPUT: '+str(d))
    plt.pcolormesh(LX,LY, 10*np.log10(LZs[di,:,:]), shading='auto',cmap='gnuplot2',vmin=0,vmax=-60)
    plt.legend()
    plt.colorbar()
    plt.axis("equal")
    plt.show()

# 3. Look at FWHMs, ellipticities

In [ ]:

fig = plt.figure(figsize=(16,8))
for d in pols: 
    ell = (fits['G_popt'][d,:,2]-fits['G_popt'][d,:,4]) / (fits['G_popt'][d,:,2]+fits['G_popt'][d,:,4])
    plt.plot(freqs, ell, marker='.',linestyle='None',label=str(d))
    plt.ylabel('Ellipticity')
    plt.xlabel('Frequency [MHz] ')
    plt.ylim(-0.2,0.2)
plt.legend()
plt.show()

print(freqs[freq_i])

# 4. Examine radial profiles

In [ ]:
## Dumb thing first: convert all centroid corrected X, Y
# to r coordinate and overplot Gaussian 


for di in range(0,len(pols)):
    d = pols[di]

    xp,yp = get_newpointing(x,y,pconcat.G_popt[d,freq_i,1],pconcat.G_popt[d,freq_i,3],0.0)  
    
    Gval = Gauss_2d_cent(xp,yp,pconcat.G_popt[d,freq_i,0],
                         pconcat.G_popt[d,freq_i,2],pconcat.G_popt[d,freq_i,4],pconcat.G_popt[d,freq_i,5])
    r = np.sqrt(xp**2 + yp**2)
    plt.plot(r,pconcat.V_bgsub[indsuse0,freq_i,d],'b,')
    xslice = get_slice(xp,yp,'y',0.0,2.0)
    plt.plot(r[xslice],pconcat.V_bgsub[np.array(indsuse0)[xslice.astype(int)],freq_i,d],'r.')
    plt.plot(r,Gval,color='black')
    plt.title('INPUT: '+str(d))
    
    plt.show()


In [ ]:
for di in range(0,len(pols)):
    d = pols[di]
    plt.plot(10*np.log10(RPs[di,:]),label=str(d))
plt.xlim(0,60)
plt.ylim(-30,0)
plt.legend()
plt.show()

In [ ]:
for di in range(0,len(pols)):
    d = pols[di]
    plt.plot(RPs[di,:],label=str(d))
plt.xlim(0,15)
plt.ylim(0,0.9)
plt.legend()
plt.show()

In [ ]:
# 4. Examine Frequency dependence

LZs = np.zeros([len(freqs),len(pols),len(LX[:,0]),len(LY[0,:])])
RPs = np.zeros([len(freqs),len(pols),142])

for di in range(0,len(pols)):
    d = pols[di]
    for f in np.arange(0,len(freqs)):
        xc = pconcat.G_popt[d,f,1] # x centroid
        yc = pconcat.G_popt[d,f,3] # y centroid
        amp_corr = pconcat.V_bgsub[indsuse0,f,d]/pconcat.G_popt[d,f,0]
                
        xp,yp = get_newpointing(x,y,xc,yc,0.0)       

        try:
            LZs[f,di,:,:] = griddata((xp,yp), amp_corr, (LX,LY), method='linear')
            RPs[f,di,:] = radial_profile(LX,LY,LZs[f,di,:,:],0,0)
        except: 'Could not linearly interpolate!!!!!'
    

In [ ]:
di = 0

F = len(freqs)
cm = plt.get_cmap('Spectral')
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.set_prop_cycle(color=[cm(1.*i/F) for i in range(F)])

for f in range(0,len(freqs)):
    plt.plot(RPs[f,di,:],alpha=0.2)
plt.xlim(0,14)
plt.show()

In [ ]:
for di in range(0,len(pols)):
    plt.imshow(10*np.log10(RPs[:,di,:]),vmin=-30,vmax=0,extent=([0, 142, 800, 400]),aspect='auto')
    plt.colorbar()
    plt.xlim(0,60)
    plt.show()

    plt.imshow(RPs[:,di,:],vmin=0,vmax=0.9,extent=([0, 142, 800, 400]),aspect='auto')
    plt.colorbar()
    plt.xlim(0,14)
    plt.show()